In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
img = mpimg.imread('/kaggle/input/image-classification-2/train/Image_003228.jpg')
plt.imshow(img)

In [3]:
img.shape

In [4]:
from skimage.color import rgb2gray
gray_image = rgb2gray(img)
plt.imshow(gray_image)

In [5]:
gray_image.shape

In [6]:
gray_image

In [7]:
flat_image = gray_image.ravel()

In [8]:
flat_image

In [9]:
flat_image.shape

In [10]:
plt.imshow(flat_image.reshape(1400, 1117))

In [11]:
img2 = mpimg.imread('/kaggle/input/image-classification-2/train/Image_018022.jpg')
plt.imshow(img2)

In [12]:
img2.shape

In [13]:
import tensorflow as tf
from tensorflow import keras
from skimage.io import imsave
import shutil

In [14]:
df_train = pd.read_csv('../input/image-classification-2/train_label.csv')

In [15]:
df_train.head()

In [16]:
df_train.label.value_counts()

In [17]:
BASE_DIR = '../input/image-classification-2/train/'
len(os.listdir(BASE_DIR))

In [18]:
!mkdir sample

In [19]:
for name in range(0,11):
    os.makedirs('./sample/'+str(name))

In [20]:
os.listdir('./sample')

In [21]:
os.listdir('./sample')[0]

In [22]:
os.path.join('./sample', str(0))

In [23]:
df_train.label[0] == int(os.path.basename('./sample/8'))

In [24]:
os.path.basename('./sample/4')

In [25]:
len(os.listdir('./sample/1/'))

In [26]:
for index in df_train.index:
    for folder_name in range(0,11):
        if df_train.loc[index, 'label'] == int(os.path.basename('./sample/'+str(folder_name))):
#             file = mpimg.imread(BASE_DIR+df_train.loc[index, 'file_name'])
#             imsave(os.path.join('./sample/'+str(folder_name)+'/',df_train.loc[index, 'file_name']), rgb2gray(file))
            shutil.copy(BASE_DIR+df_train.loc[index, 'file_name'], os.path.join('./sample/'+str(folder_name)+'/'))

In [27]:
NEW_BASE = './sample/'

In [28]:
os.makedirs('./sample/train')
os.makedirs('./sample/test')
os.makedirs('./sample/val')

In [29]:
for i in range(0,11):
    os.makedirs('./sample/train/'+str(i))
    os.makedirs('./sample/test/'+str(i))
    os.makedirs('./sample/val/'+str(i))

In [30]:
os.listdir('./sample/train/')

In [31]:
folder_list = [str(i)+'/' for i in range(0,11)]
folder_list

In [32]:
NEW_BASE = './sample/'

In [33]:
for folder_idx, folder in enumerate(folder_list):
    files = os.listdir(NEW_BASE + folder)
    folder_name = folder.split('/')[0]
    number_of_images = len([name for name in files])
    n_train = int((number_of_images * 0.6) + 0.5)
    n_valid = int((number_of_images*0.25) + 0.5)
    n_test = number_of_images - n_train - n_valid
    print(number_of_images, n_train, n_valid, n_test)
    for idx, file in enumerate(files):
        file_name = NEW_BASE + folder + file
        if idx < n_train:
            shutil.move(file_name, NEW_BASE + "train/" + folder_name)
        elif idx < n_train + n_valid:
            shutil.move(file_name, NEW_BASE + "val/" + folder_name)
        else:
            shutil.move(file_name, NEW_BASE + "test/" + folder_name)

In [34]:
553+231+138

In [35]:
len(os.listdir('./sample/train/1'))

In [39]:
for i in range(0,11):
    os.rmdir('./sample/'+str(i))

In [40]:
name_list = [str(i) for i in range(0, 11)]
name_list

In [42]:
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
vgg_model.summary()

In [43]:
model = keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    model.add(layer)

In [44]:
for layer in model.layers:
    layer.trainable = False
model.summary()

In [46]:
model.add(keras.layers.Dense(11))

In [47]:
model.summary()

In [48]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(learning_rate=0.01)
metrics = ["accuracy"]

model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [49]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [51]:
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_gen.flow_from_directory(
    './sample/train',
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    color_mode="rgb",
    classes=name_list   
)

val_batches = valid_gen.flow_from_directory(
    './sample/val',
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    color_mode="rgb",
    classes=name_list
)

test_batches = test_gen.flow_from_directory(
    './sample/test',
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=32,
    shuffle=False,
    color_mode="rgb",
    classes=name_list
)

In [52]:
epochs = 20

# early_stopping = keras.callbacks.EarlyStopping(
#     monitor="val_loss",
#     patience=5,
#     verbose=2
# )

history = model.fit(train_batches, validation_data=val_batches,
          # callbacks=[early_stopping],
          epochs=epochs, verbose=2)

In [54]:
model.save('model.h5')

In [55]:
model.evaluate(test_batches, verbose=2)

In [61]:
plt.imshow(test_batches[0][0][1].reshape(224,224,3))

In [67]:
test_batches[100][1]

In [68]:
plt.imshow(test_batches[100][0][0].reshape(224,224,3))

In [69]:
pred1 = model.predict(test_batches[100][0][0].reshape(1, 224,224,3))
pred1 = tf.nn.softmax(pred1)
label1 = np.argmax(pred1, axis=1)
label1

In [80]:
test_gen_new = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_preprocessed = test_gen_new.flow_from_directory(
    '../input/image-classification-2',
    class_mode='sparse',
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    classes=['test']
)

In [81]:
prediction_new = model.predict(test_preprocessed)

In [82]:
prediction_new = tf.nn.softmax(prediction_new)
label_new = np.argmax(prediction_new, axis=1)

In [83]:
submission = pd.read_csv('../input/image-classification-2/sample_submission.csv')
submission.head()

In [84]:
submission['label'] = label_new

In [85]:
submission.head()

In [87]:
submission.to_csv("submission.csv", index=False, header=True)